<div style="display: flex; align-items: center; justify-content: center; flex-wrap: wrap;">
    <div style="flex: 1; max-width: 400px; display: flex; justify-content: center;">
        <img src="https://i.ibb.co/JBPWVYR/Logo-Nova-IMS-Black.png" style="max-width: 50%; height: auto; margin-top: 50px; margin-bottom: 50px;margin-left: 6rem;">
    </div>
    <div style="flex: 2; text-align: center; margin-top: 20px;margin-left: 8rem;">
        <div style="font-size: 28px; font-weight: bold; line-height: 1.2;">
            <span style="color: #22c1c3;">DL Project |</span> <span style="color: #08529C;">Predicting Rare Species from Images using Deep Learning</span>
        </div>
        <div style="font-size: 17px; font-weight: bold; margin-top: 10px;">
            Spring Semester | 2024 - 2025
        </div>
        <div style="font-size: 17px; font-weight: bold;">
            Master in Data Science and Advanced Analytics
        </div>
        <div style="margin-top: 20px;">
            <div>André Silvestre, 20240502</div>
            <div>Diogo Duarte, 20240525</div>
            <div>Filipa Pereira, 20240509</div>
            <div>Maria Cruz, 20230760</div>
            <div>Umeima Mahomed, 20240543</div>
        </div>
        <div style="margin-top: 20px; font-weight: bold;">
            Group 37
        </div>
    </div>
</div>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 1px; color: white; border-radius: 500px; text-align: center;">
</div>

## **📚 Libraries Import**

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# System imports
import os
import sys
from tqdm import tqdm
from typing_extensions import Self, Any      # For Python 3.10
# from typing import Self, Any               # For Python >3.11

from pathlib import Path

# Data manipulation imports
import numpy as np
import pandas as pd  
import warnings
warnings.filterwarnings("ignore")

# Data visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning imports
import tensorflow as tf
from keras.ops import add
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Image processing imports (Data Augmentation)
from keras.layers import Rescaling, RandomBrightness, RandomFlip, RandomRotation, 
from keras.layers import Pipeline

# Evaluation imports
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
from keras.metrics import CategoricalAccuracy, AUC, F1Score

# Other imports
from itertools import product

# Image processing imports
from matplotlib.image import imread
from PIL import Image

# Set the style of the visualization
pd.set_option('future.no_silent_downcasting', True)   # use int instead of float in DataFrame
pd.set_option("display.max_columns", None)            # display all columns

# Disable warnings (FutureWarning)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# For better resolution plots
%config InlineBackend.figure_format = 'retina'
# Setting seaborn style
sns.set_theme(style="white")

# Set random seed for reproducibility
np.random.seed(2025)

## **🧮 Import Databases**

In [12]:
# Define the path to the data
train_dir = Path("data/train")
val_dir = Path("data/val")
test_dir = Path("data/test")

In [ ]:
# Image Generators
n_classes = 202                                     # Number of classes (we already know this based on previous notebook)
image_size = (224, 224)                             # Size of the images
img_width, img_height = 224, 224               
batch_size = 32                                     # Batch size
input_shape = (img_width, img_height, 3)            # Input shape of the model
value_range = (0.0, 1.0)                            # Range of pixel values

# Train data generator
train_datagen = image_dataset_from_directory(
    train_dir,                                      # Path to the directory
    labels='inferred',                              # Type of labels to generate (inferred = from the directory structure)
    label_mode='categorical',                       # Type of labels to generate (categorical = 'float32' tensor of shape (batch_size, num_classes), representing a one-hot encoding of the class index.)
    class_names=None,                               # List of class names
    color_mode='rgb',                               # Color mode to read images
    batch_size=batch_size,                          # Size of the batches of data
    image_size=(img_width, img_height),             # Size of the images to read
    shuffle=True,                                   # Whether to shuffle the data
    seed=2025,                                      # Random seed for shuffling and transformations
    validation_split=None,                          # Fraction of images reserved for validation (between 0 and 1)
    subset=None,                                    # Subset of data ('training' or 'validation') if validation_split is set
    interpolation='bilinear',                       # Interpolation method to resample the image
    follow_links=False,                             # Whether to follow symlinks inside class subdirectories
)

# Validation data generator
val_datagen = image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=2025,
    interpolation='bilinear',
    follow_links=False,
)

# Test data generator
test_datagen = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=2025,
    interpolation='bilinear',
    follow_links=False,
)

Found 8388 files belonging to 202 classes.
Found 1797 files belonging to 202 classes.
Found 1798 files belonging to 202 classes.


# <a class='anchor' id='3'></a>
<br>
<style>
@import url('https://fonts.cdnfonts.com/css/avenir-next-lt-pro?styles=29974');
</style>

<div style="background: linear-gradient(to right, #22c1c3, #27b1dd, #2d9cfd, #090979); 
            padding: 10px; color: white; border-radius: 300px; text-align: center;">
    <center><h1 style="margin-left: 140px;margin-top: 10px; margin-bottom: 4px; color: white;
                       font-size: 32px; font-family: 'Avenir Next LT Pro', sans-serif;">
        <b>3 | Modeling - Baseline Model</b></h1></center>
</div>

<br><br>

# **💡 Modeling**

#### **Data Augmentation | Pipeline**

In [ ]:
# https://keras.io/api/layers/preprocessing_layers/image_preprocessing/
augmentation_layer = Pipeline(
    [
        Rescaling(1./255),
        RandomFlip("horizontal_and_vertical"),
        RandomRotation(factor=0.1, fill_mode="reflect")
    ],
    name="augmentation_layer"
)

#### **Modeling | Baseline Model**

In [ ]:
class MyTinyCNN(Model):
    """
    MyTinyCNN class, inherets from keras' Model class
    """

    def __init__(self: Self, activation: str = "relu") -> None:
        """
        Initialization
        """

        super().__init__(name="my_tiny_oo_cnn")

        if image_size:
            self.augmentation_layer = augmentation_layer

        self.conv_layer_1 = Conv2D(
            filters=3 * 8,
            kernel_size=(3, 3),
            activation=activation,
            name="conv_layer_1"
        )
        self.max_pool_layer_1 = MaxPooling2D(
            pool_size=(2, 2),
            name="max_pool_layer_1"
        )

        # exemplify non-sequential nature of computation possible with
        # the functional and object-oriented methods
        self.conv_layer_2l = Conv2D(
            filters=3 * 16,
            kernel_size=(3, 3),
            activation=activation,
            name="conv_layer_2l",
            padding="same"
        )
        self.conv_layer_2r = Conv2D(
            filters=3 * 16,
            kernel_size=(2, 2),
            activation=activation,
            name="conv_layer_2r",
            padding="same"
        )
        self.max_pool_layer_2 = MaxPooling2D(
            pool_size=(2, 2),
            name="max_pool_layer_2"
        )

        self.flatten_layer = Flatten(name="flatten_layer")
        self.dropout = Dropout(rate=0.3)
        self.dense_layer = Dense(
            n_classes,
            activation="softmax",
            name="classification_head"
        )

    def call(self: Self, inputs: Any) -> Any:
        """
        Forward call
        """

        x = self.augmentation_layer(inputs)


        x = self.conv_layer_1(x)
        x = self.max_pool_layer_1(x)

        # exemplify non-sequential nature of computation possible with
        # the functional and object-oriented methods
        x_l = self.conv_layer_2l(x)
        x_r = self.conv_layer_2r(x)
        x = add(x_l, x_r)
        x = self.max_pool_layer_2(x)

        x = self.flatten_layer(x)
        x = self.dropout(x)

        return  self.dense_layer(x)

In [28]:
# Train our regularized MyTinyCNN
model = MyTinyCNN()
# optimizer = Adam(learning_rate=0.001)
optimizer = SGD(learning_rate=0.01, name="optimizer", weight_decay=0.01)
loss = CategoricalCrossentropy(name="loss")

In [ ]:
model.summary()

In [29]:
# Metrics
categorical_accuracy = CategoricalAccuracy(name="accuracy")
auc = AUC(name="auc")
f1_score = F1Score(average="macro", name="f1_score")
metrics = [categorical_accuracy, auc, f1_score]

In [30]:
# Traces the computation
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [31]:
# Callbacks
root_dir_path = Path(".")
checkpoint_file_path = root_dir_path / "checkpoint.keras"
metrics_file_path = root_dir_path = root_dir_path / "metrics.csv"

checkpoint_callback = ModelCheckpoint(
    checkpoint_file_path,                                             # Save for each epoch the best model
    monitor="val_loss",
    verbose=0
)
metrics_callback = CSVLogger(metrics_file_path)                       # Save for the metrics

In [32]:
# Learning Rate
def exp_decay_lr_scheduler(
    epoch: int,                                              # O 1º e o 2º argumentos têm de ser o epoch e o lr nesta ordem
    current_lr: float,
    factor: float = 0.95
) -> float:
    """
    Exponential decay learning rate scheduler
    """

    current_lr *= factor

    return current_lr

In [33]:
lr_scheduler_callback = LearningRateScheduler(exp_decay_lr_scheduler)

In [34]:
callbacks = [
    checkpoint_callback,
    metrics_callback,
    lr_scheduler_callback
]                                                   # List of callbacks

In [36]:
# train the model
_ = model.fit(train_datagen,
              epochs=10,
              validation_data=val_datagen,
              callbacks=callbacks,
              verbose=2)

Epoch 1/10


I0000 00:00:1743105608.178889    6224 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-27 20:00:27.421526: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 346800000 exceeds 10% of free system memory.


263/263 - 50s - 192ms/step - accuracy: 0.0291 - auc: 0.6490 - f1_score: 0.0022 - loss: 5.0914 - val_accuracy: 0.0273 - val_auc: 0.6653 - val_f1_score: 0.0018 - val_loss: 5.0461 - learning_rate: 0.0095
Epoch 2/10


2025-03-27 20:01:00.731029: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 346800000 exceeds 10% of free system memory.


263/263 - 34s - 129ms/step - accuracy: 0.0324 - auc: 0.6631 - f1_score: 0.0025 - loss: 5.0240 - val_accuracy: 0.0250 - val_auc: 0.6522 - val_f1_score: 0.0011 - val_loss: 5.1413 - learning_rate: 0.0090
Epoch 3/10


...

263/263 - 39s - 148ms/step - accuracy: 0.0360 - auc: 0.6731 - f1_score: 0.0040 - loss: 4.9930 - val_accuracy: 0.0356 - val_auc: 0.6747 - val_f1_score: 0.0033 - val_loss: 5.0057 - learning_rate: 0.0086
Epoch 4/10


2025-03-27 20:02:12.905797: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 346800000 exceeds 10% of free system memory.


263/263 - 32s - 122ms/step - accuracy: 0.0404 - auc: 0.6791 - f1_score: 0.0046 - loss: 4.9622 - val_accuracy: 0.0323 - val_auc: 0.6676 - val_f1_score: 0.0069 - val_loss: 5.0465 - learning_rate: 0.0081
Epoch 5/10


2025-03-27 20:02:45.734132: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 346800000 exceeds 10% of free system memory.


263/263 - 34s - 129ms/step - accuracy: 0.0380 - auc: 0.6862 - f1_score: 0.0058 - loss: 4.9425 - val_accuracy: 0.0267 - val_auc: 0.6674 - val_f1_score: 7.4905e-04 - val_loss: 5.0609 - learning_rate: 0.0077
Epoch 6/10
263/263 - 32s - 120ms/step - accuracy: 0.0439 - auc: 0.6893 - f1_score: 0.0095 - loss: 4.9207 - val_accuracy: 0.0356 - val_auc: 0.6843 - val_f1_score: 0.0085 - val_loss: 4.9800 - learning_rate: 0.0074
Epoch 7/10
263/263 - 30s - 115ms/step - accuracy: 0.0420 - auc: 0.6930 - f1_score: 0.0127 - loss: 4.9052 - val_accuracy: 0.0434 - val_auc: 0.6860 - val_f1_score: 0.0126 - val_loss: 4.9823 - learning_rate: 0.0070
Epoch 8/10
263/263 - 32s - 123ms/step - accuracy: 0.0457 - auc: 0.6944 - f1_score: 0.0115 - loss: 4.8969 - val_accuracy: 0.0295 - val_auc: 0.6626 - val_f1_score: 0.0075 - val_loss: 5.1943 - learning_rate: 0.0066
Epoch 9/10
263/263 - 30s - 115ms/step - accuracy: 0.0492 - auc: 0.6996 - f1_score: 0.0143 - loss: 4.8732 - val_accuracy: 0.0245 - val_auc: 0.6747 - val_f1_scor

---

### <a class='anchor' id='3_1'></a> <a class='anchor' id='3_2'></a>  **🧪 Model Selection & 📏 Model Evaluation**

In [ ]:
# evaluate on the test set
model.evaluate(
    test_datagen,
    batch_size=batch_size,
    return_dict=True,
    verbose=0
)

{'accuracy': 0.0339265838265419,
 'auc': 0.6677749752998352,
 'f1_score': 0.006862386129796505,
 'loss': 5.02635383605957}

: 

---

---

# **🔗 Bibliography/References**

**[[1]](https://)** AAAAAAAAAA

---